# Superstore → Sheets → Looker Studio  
Clean → KPIs → Export (Colab)
Purpose: build a reproducible pipeline that cleans Superstore data, computes KPIs, and publishes to Google Sheets for Looker Studio.


In [1]:
# --- 0) Installs (quiet) ---
!pip -q install pandas gspread gspread-dataframe google-auth google-auth-oauthlib

## Auth
Authorize Colab to access your Google Drive/Sheets.


In [2]:
# --- 1) Auth (Colab → Google) ---
from google.colab import auth
auth.authenticate_user()

## Imports
Core libs for data, auth, and Sheets I/O.

In [3]:
# --- 2) Imports ---
import pandas as pd
import numpy as np
import gspread
from google.auth import default
from gspread_dataframe import get_as_dataframe, set_with_dataframe

## Config
Set your Google Sheet URL and tab names.

In [4]:
# --- 3) Config (edit these) ---
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1dYaYdDxD8fP4z65D5kiRLM8Oi7fNUXPKt0Ey3j5JgVM/edit?usp=sharing"
RAW_TAB        = "Superstore"                    # input tab (raw)
PREP_TAB       = "Superstore - Prep"             # cleaned output tab
KPI_TAB        = "Superstore - KPI_Summary"      # KPI output tab

## Connect to Google Sheets
Create a client and open the spreadsheet by URL.

In [5]:
# --- 4) Connect to the Google Sheet ---
creds, _ = default()
gc = gspread.authorize(creds)
sh = gc.open_by_url(SPREADSHEET_URL)

## Load raw data
Read the raw Superstore tab into a DataFrame.

In [6]:
# --- 5) Load raw data from RAW_TAB ---
ws_raw = sh.worksheet(RAW_TAB)
df = get_as_dataframe(ws_raw, evaluate_formulas=True, keep_empty_rows=False)

## Canonicalize columns and drop empties
Tighten headers and remove fully empty rows/cols.

In [7]:
# --- 6) Canonicalization ---
df.columns = [c.strip() for c in df.columns]
df = df.dropna(how="all")
df = df.dropna(axis=1, how="all")

## Schema sanity
Warn if expected columns are missing.

In [8]:
# --- 7) Expected columns check ---
expected = {
    "Order ID","Order Date","Ship Date","Ship Mode","Customer ID","Customer Name",
    "Segment","Country","City","State","Postal Code","Region",
    "Product ID","Category","Sub-Category","Sales","Quantity","Discount","Profit"
}
missing = [c for c in expected if c not in df.columns]
if missing:
    print("⚠ Missing expected columns:", missing)

## Type fixes
Dates → datetime, numeric fields → floats, geo text cleaned.

In [9]:
# --- 8) Type fixes ---
for col in ["Order Date","Ship Date"]:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")

for col in ["Sales","Profit","Discount"]:
    if col in df.columns:
        df[col] = (
            df[col].astype(str)
                   .str.replace(r"[\$,]", "", regex=True)
                   .replace({"": np.nan})
                   .astype(float)
        )

if "Quantity" in df.columns:
    df["Quantity"] = pd.to_numeric(df["Quantity"], errors="coerce")

if "State" in df.columns:
    df["State"] = df["State"].astype(str).str.strip()


## Derived fields
Create helper time keys and profitability metrics.

In [10]:
# --- 9) Derived fields ---
if "Order Date" in df.columns:
    df["Order_Year"]  = df["Order Date"].dt.year
    df["Order_Month"] = df["Order Date"].dt.to_period("M").astype(str)  # YYYY-MM

if {"Sales","Profit"}.issubset(df.columns):
    df["Profit_Margin"] = np.where(df["Sales"] > 0, df["Profit"]/df["Sales"], np.nan)

if {"Sales","Order ID"}.issubset(df.columns):
    df["AOV_helper"] = df["Sales"]  # used for aggregate AOV later

## Quick data quality scan
Null counts to spot problems fast.

In [11]:
# --- 10) Null report ---
def quick_null_report(frame):
    n = len(frame)
    nz = frame.isna().sum().sort_values(ascending=False)
    return pd.DataFrame({"nulls": nz, "null_%": (nz/n*100).round(2)})

dq = quick_null_report(df)
print("Null report (top 10):")
display(dq.head(10))

Null report (top 10):


,nulls,null_%
Row ID,0,0.0
Order ID,0,0.0
Order Date,0,0.0
Ship Date,0,0.0
Ship Mode,0,0.0
Customer ID,0,0.0
Customer Name,0,0.0
Segment,0,0.0
Country,0,0.0
City,0,0.0


## KPI aggregation
Compute totals and formatted strings for the KPI tab.

In [12]:
# --- 11) Aggregate KPIs ---
total_sales   = float(df["Sales"].sum())        if "Sales"    in df else np.nan
total_profit  = float(df["Profit"].sum())       if "Profit"   in df else np.nan
total_orders  = df["Order ID"].nunique()        if "Order ID" in df else np.nan
profit_margin = (total_profit/total_sales) if total_sales else np.nan
aov           = (total_sales/total_orders) if total_orders else np.nan

kpi_summary = pd.DataFrame(
    {
        "KPI": ["Total Sales","Total Profit","Total Orders","Profit Margin %","Avg Order Value"],
        "Value": [total_sales, total_profit, total_orders,
                  profit_margin*100 if pd.notna(profit_margin) else np.nan, aov],
        "Formatted": [
            pd.Series([total_sales]).map("${:,.2f}".format).iat[0] if pd.notna(total_sales) else "",
            pd.Series([total_profit]).map("${:,.2f}".format).iat[0] if pd.notna(total_profit) else "",
            f"{int(total_orders):,}" if pd.notna(total_orders) else "",
            f"{profit_margin*100:,.2f}%" if pd.notna(profit_margin) else "",
            pd.Series([aov]).map("${:,.2f}".format).iat[0] if pd.notna(aov) else "",
        ],
    }
)
kpi_summary

,KPI,Value,Formatted
0,Total Sales,2.297201e+06,"$2,297,200.86"
1,Total Profit,2.863970e+05,"$286,397.02"
2,Total Orders,5.009000e+03,"5,009"
3,Profit Margin %,1.246722e+01,12.47%
4,Avg Order Value,4.586147e+02,$458.61


## Breakdown tables
Yearly and category-level summaries for validation and optional export.

In [13]:
# --- 12) Year and Category breakdowns ---
grp_year = pd.DataFrame()
if "Order_Year" in df.columns:
    grp_year = (
        df.groupby("Order_Year")
          .agg(Sales=("Sales","sum"), Profit=("Profit","sum"), Orders=("Order ID","nunique"))
          .reset_index()
          .sort_values("Order_Year")
    )

grp_cat = pd.DataFrame()
if {"Category","Sales","Profit"}.issubset(df.columns):
    grp_cat = (
        df.groupby("Category")
          .agg(Sales=("Sales","sum"), Profit=("Profit","sum"))
          .assign(Profit_Margin=lambda x: np.where(x["Sales"]>0, x["Profit"]/x["Sales"], np.nan))
          .reset_index()
          .sort_values("Sales", ascending=False)
    )

display(grp_year.head())
display(grp_cat.head())

,Order_Year,Sales,Profit,Orders
0,2014,484247.4981,49543.9741,969
1,2015,470532.5090,61618.6037,1038
2,2016,609205.5980,81795.1743,1315
3,2017,733215.2552,93439.2696,1687


,Category,Sales,Profit,Profit_Margin
2,Technology,836154.0330,145454.9481,0.173957
0,Furniture,741999.7953,18451.2728,0.024867
1,Office Supplies,719047.0320,122490.8008,0.170352


## Sheets helpers
Upsert worksheet: create if absent, otherwise clear for fresh write.

In [15]:
# --- 13) Upsert worksheet helper ---
def upsert_worksheet(spreadsheet, title, rows=1000, cols=26):
    try:
        ws = spreadsheet.worksheet(title)
        ws.clear()
    except gspread.WorksheetNotFound:
        ws = spreadsheet.add_worksheet(title=title, rows=str(rows), cols=str(cols))
    return ws

## Publish cleaned data
Write the cleaned DataFrame to the PREP tab for Looker Studio.

In [16]:
# --- 14) Write cleaned data to PREP_TAB ---
ws_prep = upsert_worksheet(sh, PREP_TAB, rows=max(len(df)+10, 1000), cols=max(len(df.columns)+2, 26))
set_with_dataframe(ws_prep, df)
print(f"✅ Wrote cleaned data to '{PREP_TAB}' ({len(df):,} rows, {len(df.columns)} cols).")

✅ Wrote cleaned data to 'Superstore - Prep' (9,994 rows, 25 cols).


## Publish KPIs and summaries
Write KPI table, then append Year and Category summaries.

In [17]:
# --- 15) Write KPI summary + breakdowns to KPI_TAB ---
ws_kpi = upsert_worksheet(sh, KPI_TAB, rows=2000, cols=10)

# Page 1: KPIs
set_with_dataframe(ws_kpi, kpi_summary)

# Spacer row
ws_kpi.append_row([])

# Page 2: Yearly summary
if not grp_year.empty:
    ws_kpi.append_row(["Yearly Summary"])
    set_with_dataframe(ws_kpi, grp_year, row=ws_kpi.row_count)

# Spacer row
ws_kpi.append_row([])

# Page 3: Category summary
if not grp_cat.empty:
    ws_kpi.append_row(["Category Summary"])
    set_with_dataframe(ws_kpi, grp_cat, row=ws_kpi.row_count)

print(f"✅ Wrote KPIs + breakdowns to '{KPI_TAB}'.")

✅ Wrote KPIs + breakdowns to 'Superstore - KPI_Summary'.


## Final notes for Looker Studio
In the data source schema, set:
- `State` → **Geo → Country subdivision (1st level)**
- `Order Date` → **Date**
Use **Superstore - Prep** as the main source for visuals.